## generate abilene training dataset for TELGEN

In [6]:
from solver.linprog import linprog
from tqdm import tqdm

import gzip
import pickle
import torch
from scipy.linalg import LinAlgWarning
from scipy.optimize._optimize import OptimizeWarning
# from scipy.optimize import OptimizeWarning
import warnings
import numpy as np
from functools import partial
import random
import pickle 
import json

In [7]:
root = 'raw/'

### Resource Allocation

#### input: for one graph   
#### G(V, E, c): random graph (strongly connected)
#### (s, t, d) \in [S, T, D]  
#### for every (s, t, d), there is a set p \in Pd (k-shortest path algorithm (4/5/6))

In [8]:
import networkx as nx
import matplotlib.pyplot as plt

### generate and save connected and directed ASN different nodes and p

In [9]:
f = open(root+'abilene_graph/ABILENE.json')
G = json.load(f)

abilene_graph = nx.DiGraph()
abilene_graph.add_nodes_from([i['id'] for i in G['nodes']])
abilene_graph.add_edges_from([(i['source'], i['target']) for i in G['links']])

print('Strongly connected:', nx.is_strongly_connected(abilene_graph))
print('# of nodes and edges:', abilene_graph.number_of_nodes(), abilene_graph.number_of_edges())
print('Weighted:', nx.is_weighted(abilene_graph))

abilene_graph_weight = nx.DiGraph()
abilene_graph_weight.add_nodes_from([i['id'] for i in G['nodes']])
for i in G['links']:
    abilene_graph_weight.add_edge(i['source'], i['target'], weight=float(i['capacity']))
print('Strongly connected:', nx.is_strongly_connected(abilene_graph_weight))
print('# of nodes and edges:', abilene_graph_weight.number_of_nodes(), abilene_graph_weight.number_of_edges())
print('Weighted:', nx.is_weighted(abilene_graph_weight))

# nx.draw(asn_graph, with_labels=True, node_color='lightgreen', arrows=True)

Strongly connected: True
# of nodes and edges: 12 30
Weighted: False
Strongly connected: True
# of nodes and edges: 12 30
Weighted: True


In [10]:
# check b4 link capacities
cap = []
for i in G['links']:
    cap.append(float(i['capacity']))
max(cap), min(cap), type(cap[0])

(9.92, 2.48, float)

### generate k-shortest path

In [11]:
from itertools import islice
def k_shortest_paths(G, source, target, k, weight=None):
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))

### Read abilene and their capacities and load as a group

In [12]:
root = 'raw/'
f = open(root+'abilene_graph/ABILENE.json')
G = json.load(f)

abilene_graph = nx.DiGraph()
abilene_graph.add_nodes_from([i['id'] for i in G['nodes']])
abilene_graph.add_edges_from([(i['source'], i['target']) for i in G['links']])

abilene_graph_weight = nx.DiGraph()
abilene_graph_weight.add_nodes_from([i['id'] for i in G['nodes']])
for i in G['links']:
    abilene_graph_weight.add_edge(i['source'], i['target'], weight=float(i['capacity']))

group = []
group_noC = []
group.append(abilene_graph_weight)
group_noC.append(abilene_graph)

### training st pairs
### --train_start_indices 0 --train_end_indices 12096  (1-12095)
### --val_start_indices 12096 --val_end_indices 14112 (12096-14111)
### --test_start_idx 14112 --test_end_idx 16128 (14112-16128)

In [13]:
# root = '/mnt/data/fzhou/HARP/'
# path = root+'traffic_matrices/abilene/t'
# ######################## training ########################
# n = subgraph.number_of_nodes()
# STD = []
# for i in range(1, 12096):
#     with open(path+str(i)+'.pkl', 'rb') as f:
#         data = pickle.load(f)
#     std = []
#     for i in range(n):
#         for j in range(n):
#             if i == j:
#                 continue
#             elif i > j:
#                 std.append((i, j, data[(n-1)*i+j]))
#             else:       # i<j
#                 std.append((i, j, data[(n-1)*i+j-1]))
#     std = np.array(std)
#     STD.append(std)
# STD = np.array(STD)
# print('train size', STD.shape)
# np.save('abilene_train', STD)
# ######################## validation ########################
# STD = []
# for i in range(12096, 14112):
#     with open(path+str(i)+'.pkl', 'rb') as f:
#         data = pickle.load(f)
#     std = []
#     for i in range(n):
#         for j in range(n):
#             if i == j:
#                 continue
#             elif i > j:
#                 std.append((i, j, data[(n-1)*i+j]))
#             else:       # i<j
#                 std.append((i, j, data[(n-1)*i+j-1]))
#     std = np.array(std)
#     STD.append(std)
# STD = np.array(STD)
# print('valid size', STD.shape)
# np.save('abilene_valid', STD)
# ######################## testing ########################
# STD = []
# for i in range(14112, 16128):
#     with open(path+str(i)+'.pkl', 'rb') as f:
#         data = pickle.load(f)
#     std = []
#     for i in range(n):
#         for j in range(n):
#             if i == j:
#                 continue
#             elif i > j:
#                 std.append((i, j, data[(n-1)*i+j]))
#             else:       # i<j
#                 std.append((i, j, data[(n-1)*i+j-1]))
#     std = np.array(std)
#     STD.append(std)
# STD = np.array(STD)
# print('test size', STD.shape)
# np.save('abilene_test', STD)

In [14]:
# size: (# of instance, # of pairs, std values)
train_std = np.load('abilene_train.npy')
print('train size', train_std.shape)
valid_std = np.load('abilene_valid.npy')
print('valid size', valid_std.shape)
test_std = np.load('abilene_test.npy')
print('test size', test_std.shape)

train size (12095, 132, 3)
valid size (2016, 132, 3)
test size (2016, 132, 3)


In [15]:
# # get path dic for all pairs at one time

# k = 8  
# Pd = {}
# for m in range(STD.shape[1]): 
#     st = STD[0][m][0:2]
#     k_paths = k_shortest_paths(group_noC[0], int(st[0]), int(st[1]), k=k)
#     if len(k_paths) != k:
#         for i in range(8-len(k_paths)):
#             k_paths.append([-1, -1])
#     Pd[(st[0], st[1])] = k_paths
# Pd

In [16]:
# get path dic for all pairs at one time

k = 8  
Pd = {}
for m in range(train_std.shape[1]): 
    st = train_std[0][m][0:2]
    k_paths = k_shortest_paths(group_noC[0], int(st[0]), int(st[1]), k=k)
    if len(k_paths) != k:
        for i in range(8-len(k_paths)):
            k_paths.append(k_paths[0])
    Pd[(st[0], st[1])] = k_paths
Pd

{(0.0, 1.0): [[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]],
 (0.0, 2.0): [[0, 1, 5, 2],
  [0, 1, 11, 8, 2],
  [0, 1, 4, 6, 5, 2],
  [0, 1, 4, 7, 9, 3, 6, 5, 2],
  [0, 1, 4, 7, 9, 10, 3, 6, 5, 2],
  [0, 1, 5, 2],
  [0, 1, 5, 2],
  [0, 1, 5, 2]],
 (0.0, 3.0): [[0, 1, 4, 6, 3],
  [0, 1, 5, 6, 3],
  [0, 1, 4, 7, 9, 3],
  [0, 1, 4, 7, 9, 10, 3],
  [0, 1, 11, 8, 2, 5, 6, 3],
  [0, 1, 5, 6, 4, 7, 9, 3],
  [0, 1, 5, 6, 4, 7, 9, 10, 3],
  [0, 1, 11, 8, 2, 5, 6, 4, 7, 9, 3]],
 (0.0, 4.0): [[0, 1, 4],
  [0, 1, 5, 6, 4],
  [0, 1, 11, 8, 2, 5, 6, 4],
  [0, 1, 5, 6, 3, 9, 7, 4],
  [0, 1, 5, 6, 3, 10, 9, 7, 4],
  [0, 1, 11, 8, 2, 5, 6, 3, 9, 7, 4],
  [0, 1, 11, 8, 2, 5, 6, 3, 10, 9, 7, 4],
  [0, 1, 4]],
 (0.0, 5.0): [[0, 1, 5],
  [0, 1, 4, 6, 5],
  [0, 1, 11, 8, 2, 5],
  [0, 1, 4, 7, 9, 3, 6, 5],
  [0, 1, 4, 7, 9, 10, 3, 6, 5],
  [0, 1, 5],
  [0, 1, 5],
  [0, 1, 5]],
 (0.0, 6.0): [[0, 1, 4, 6],
  [0, 1, 5, 6],
  [0, 1, 4, 7, 9, 3, 6],
  [0, 1, 11, 8, 2, 5, 6],
  [0, 1, 4, 7, 9, 10,

In [17]:
# G: G(V, E, C)                           nx.weighted.graph
# STD: demands align with ST pairs        list[([s1, t1], dmd1), ([s2, t2], dmd2),...], (string, int)
# Pd: set of paths for every st pair      dict{[s1, t1]: [([path1], cost1), ([path2], cost2)...], [s2, t2]...}
# # of std pairs = # of keys in Pd
# k: k shortest path for every (s, t, d) tuple
# MLU problem formulation:
# obj: MLU = max { \max_{p\in e} [\sum_{d\in D} d*x_d(p)] / c_e }, \forall e\in E
# s.t.: \sum_{p\in Pd} x_d(p) <= 1, \forall d \in D
#       \sum_{p\in e} \sum_{d\in D} d*x_d(p) <= c_e*MLU, \forall e\in E

# number of vars: len(STD)*k+1 

def generate_reallocation(G, STD, Pd, k):
    
    # constraint 1
    A1 = []
    for i in range(len(STD)):
        a = np.zeros(len(STD)*k+1)
        a[k*i: k*i+k] = 1
        A1.append(a)
    A1 = np.array(A1)
    b1 = np.ones(len(STD))

    # constrain 2
    edges_list = list(G.edges())                          
    A2 = np.zeros((G.number_of_edges(), len(STD)*k+1))

    for i in range(len(STD)):          # number of st pairs
        paths = Pd[tuple(STD[i][0:2])] # STD[i][0:2]: st pair; Pd[tuple(STD[i][0:2])]: possible paths
        for j in range(k):
            p = paths[j]   # path[j] is the path
            for n in range(len(p)-1):
                if (p[n], p[n+1]) in edges_list:   
                    A2[edges_list.index((p[n], p[n+1]))][k*i+j] = STD[i][-1] # STD[i][-1]: demand
                else:
                    continue  
    A2[:, -1] = -np.array(list(nx.get_edge_attributes(G,'weight').values()))
    for i in range(A2.shape[0]):      # normalize by the weight
        A2[i] = A2[i]/-A2[i, -1]
    b2 = np.zeros(G.number_of_edges())   

    zero_row_indices = np.where(A2.any(axis=1)==0)[0]
    A2 = np.delete(A2, zero_row_indices, axis=0)
    b2 = np.delete(b2, zero_row_indices, axis=0)
    
    # obj
    c = np.zeros(len(STD)*k+1)
    c[-1] = 1

    return A1, b1, A2, b2, c

## dataset generation

In [18]:
### gen train ####
root = 'raw/'

import time
warnings.filterwarnings("error")

random.seed(2024)
np.random.seed(2024)


pkg_idx = 0              # instance index for your data generation
success_cnt = 0
fail_cnt = 0
bounds = (0., None)
max_iter = 15000


data_t = 'train'        # 'train'/'valid'/test'


if data_t == 'train': 
    STD = train_std[0:10]
elif data_t == 'valid':
    STD = valid_std
else:
    STD = test_std
# STD size: (# of instance, # of pairs, std values)   

MLU = []
graph_info = []
for g in range(len(group)):
    stds = []
    ips = []
    success_cnt = 0
    times = []
    for n in range(STD.shape[0]): # in case failsure case
        std = list(STD[n])
        
        A1, b1, A2, b2, c = generate_reallocation(group[g], std, Pd, k)
        
        n_time = time.time()
        try:
            A_eq = A1
            b_eq = b1
            A_ub = A2
            b_ub = b2
            res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, 
                          method='interior-point')
            times.append(time.time()-n_time)
            print(res)
            MLU.append(res.x[-1])
            
        except (LinAlgWarning, OptimizeWarning, AssertionError):
            fail_cnt += 1
            continue
        else:
            if res.success and not np.isnan(res.fun):
                ips.append((torch.from_numpy(A1).to(torch.float), torch.from_numpy(b1).to(torch.float), 
                            torch.from_numpy(A2).to(torch.float), torch.from_numpy(b2).to(torch.float), 
                            torch.from_numpy(c).to(torch.float)))
                success_cnt += 1
                stds.append(std)


    with open(root+'/raw/instance_'+str(pkg_idx)+'_stds.pkl','wb') as f:
        pickle.dump(stds, f)
    with gzip.open(f'{root}/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
        pickle.dump(ips, file)
    pkg_idx += 1

    graph_info.append((group[g].number_of_nodes(), group[g].number_of_edges(), sum(times)/len(times)))

if data_t == 'train': 
    np.save(root+'/raw/abilene_train_info', graph_info)
    np.save(root+'/raw/abilene_train_mlu', MLU)
    for i in graph_info:
        print('Graph info and average time used:', i)
elif data_t == 'valid':
    np.save(root+'/raw/abilene_valid_info', graph_info)
    np.save(root+'/raw/abilene_valid_mlu', MLU)
    for i in graph_info:
        print('Graph info and average time used:', i)
else: 
    np.save(root+'/raw/abilene_test_info', graph_info)
    np.save(root+'/raw/abilene_test_mlu', MLU)
    for i in graph_info:
        print('Graph info and average time used:', i)
    
    
warnings.resetwarnings()

      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3320465806618018
            x: [ 1.250e-01  1.250e-01 ...  2.873e-02  3.320e-01]
          nit: 9
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3389567881839087
            x: [ 1.250e-01  1.250e-01 ...  2.639e-02  3.390e-01]
          nit: 9
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3312115182962053
            x: [ 1.250e-01  1.250e-01 ...  1.856e-02  3.312e-01]
          nit: 9
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.33619576454504724
            x: [ 1.250e-01  1.250e-01 ...  2.253e-02  3.362e-01]
          nit: 9
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun

Exception ignored in: <_io.FileIO name='raw/abilene_graph/ABILENE.json' mode='rb' closefd=True>
Traceback (most recent call last):
  File "/tmp/ipykernel_3450396/612194404.py", line 65, in <module>


      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.30356304623868646
            x: [ 1.250e-01  1.250e-01 ...  1.939e-02  3.036e-01]
          nit: 10
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3011813752903649
            x: [ 1.250e-01  1.250e-01 ...  1.757e-02  3.012e-01]
          nit: 9
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.2951756849840674
            x: [ 1.250e-01  1.250e-01 ...  1.766e-02  2.952e-01]
          nit: 10
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: 0.3141531152267706
            x: [ 1.250e-01  1.250e-01 ...  2.223e-02  3.142e-01]
          nit: 9
 intermediate: []
Graph info and average time used: (12, 30, 0.023985576629638673)


In [34]:
A1.shape, b1.shape, A2.shape, b2.shape, c.shape

((132, 1057), (132,), (30, 1057), (30,), (1057,))

In [4]:
import pickle
import gzip
import torch
import numpy as np

# with gzip.open('raw/raw/abilene_leq1/instance_2.pkl.gz', 'rb') as f:
#     data_test = pickle.load(f)
    
data_train_toy = torch.load('raw/processed_1restarts_0lap_16steps_upper_train_toy_abilene/batch0.pt')
data_train = torch.load('raw/processed_1restarts_0lap_16steps_upper_trainleq1_abilene/batch0.pt')

In [5]:
data_train_toy

HeteroDataBatch(
  gt_primals=[3171, 16],
  obj_value=[3],
  obj_const=[3171],
  A1_row=[3168],
  A1_col=[3168],
  A1_val=[3168],
  A1_num_row=[3],
  A1_num_col=[3],
  A1_nnz=[3],
  A1_tilde_mask=[3168],
  rhs1=[396],
  A2_row=[15526],
  A2_col=[15526],
  A2_val=[15526],
  A2_num_row=[3],
  A2_num_col=[3],
  A2_nnz=[3],
  A2_tilde_mask=[15526],
  rhs2=[90],
  cons={
    x=[90, 2],
    batch=[90],
    ptr=[4],
  },
  econs={
    x=[396, 2],
    batch=[396],
    ptr=[4],
  },
  vals={
    x=[3171, 2],
    batch=[3171],
    ptr=[4],
  },
  obj={
    x=[3, 2],
    batch=[3],
    ptr=[4],
  },
  (cons, to, vals)={
    edge_index=[2, 15526],
    edge_attr=[15526, 1],
  },
  (vals, to, cons)={
    edge_index=[2, 15526],
    edge_attr=[15526, 1],
  },
  (econs, to, vals)={
    edge_index=[2, 3168],
    edge_attr=[3168, 1],
  },
  (vals, to, econs)={
    edge_index=[2, 3168],
    edge_attr=[3168, 1],
  },
  (vals, to, obj)={
    edge_index=[2, 3171],
    edge_attr=[3171, 1],
  },
  (obj, to, va

In [3]:
data_train

HeteroDataBatch(
  gt_primals=[12784415, 16],
  obj_value=[12095],
  obj_const=[12784415],
  A_row=[70009995],
  A_col=[70009995],
  A_val=[70009995],
  A_num_row=[12095],
  A_num_col=[12095],
  A_nnz=[12095],
  A_tilde_mask=[70009995],
  rhs=[1959390],
  cons={
    x=[1959390, 2],
    batch=[1959390],
    ptr=[12096],
  },
  vals={
    x=[12784415, 2],
    batch=[12784415],
    ptr=[12096],
  },
  obj={
    x=[12095, 2],
    batch=[12095],
    ptr=[12096],
  },
  (cons, to, vals)={
    edge_index=[2, 70009995],
    edge_attr=[70009995, 1],
  },
  (vals, to, cons)={
    edge_index=[2, 70009995],
    edge_attr=[70009995, 1],
  },
  (vals, to, obj)={
    edge_index=[2, 12784415],
    edge_attr=[12784415, 1],
  },
  (obj, to, vals)={
    edge_index=[2, 12784415],
    edge_attr=[12784415, 1],
  },
  (cons, to, obj)={
    edge_index=[2, 1959390],
    edge_attr=[1959390, 1],
  },
  (obj, to, cons)={
    edge_index=[2, 1959390],
    edge_attr=[1959390, 1],
  }
)

In [40]:
70009995/12784415

5.476198558948532

In [220]:
# raw/raw/abilene
# instance_0/1/2: train/valid/test

In [ ]:
#python3 frameworks/gurobi_mlu.py --num_paths_per_pair 8 --opt_start_idx 0 --opt_end_idx 1 --topo abilene --framework gurobi